## Insight for the medical dataset

#### To create the dataframes and files needed for our analysis, make sure that:
    You have full_database.xml under data/medical/ folder, which will represent the XML version of the DrugBank
    You have BindingDB_All.tsv under data/medical/ folder, which will represent the tsv version of the BindingDB

### Imports

In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd

from drugbank_XML_drugparser import DrugParser
from drugbank_bindingdb_merger import DrugBank_BindingDB_Merger
from preprocessing import Preprocessing, CleanNumericAtrributesStrategy

%matplotlib inline

%load_ext autoreload
%autoreload 2

### Paths

In [4]:
DATA_PATH = 'data/medical/'
BINDINGDB = DATA_PATH + 'BindingDB_All.tsv'
DRUGBANK_XML = DATA_PATH + 'full_database.xml'
DRUGBANK_CSV = DATA_PATH + 'parsed_DrugBank.csv'
MERGED_CSV = DATA_PATH + 'Merged_Binding_DrugBank_LEFT.csv'
MERGED_PARQUET = DATA_PATH + 'merged_dataframe.parquet'
MERGED_PICKLE = DATA_PATH + 'merged_dataframe.pkl'

### Loading the data

In [5]:
def load_BindingDB(file_path):

    ## load the first row to get the number of columns
    df_first_row = pd.read_csv(file_path, sep='\t', nrows=1)

    ## number of columns to use
    df = pd.read_csv(file_path, sep='\t', header=0, usecols=range(df_first_row.shape[1]))
    return df

In [6]:
import gc
if os.path.exists(MERGED_PARQUET):
    print("Merged dataset exists.\n Loading...")

    merged_df = pd.read_pickle(MERGED_PICKLE)

    print("Merged dataset loaded")

else:
    print("Merged dataset doesn't exists.\n Creating it...")

    if os.path.exists(DRUGBANK_CSV):
        print("parsed_Drugbank exists...")
        print("Loading...")

        drugbank = pd.read_csv(DRUGBANK_CSV, encoding='utf-8')
    else:
        print("parsed_Drugbank doesn't exists...")
        print("Creating parsed_Drugbank.csv")

        drugparser = DrugParser(DRUGBANK_XML)
        drugparser.parse_drugs()
        drugbank = drugparser.save_parsed_drugs(DRUGBANK_CSV, return_df = True)

        print("parsed_Drugbank.csv is created")
        print("DrugBank XML is parsed. \n Loading it ...")

    print("Load Binding DB...")
    bindingDB_df = load_BindingDB(BINDINGDB)

    print("Cleaning Binding DB...")
    preprocessor = Preprocessing(
        [
            CleanNumericAtrributesStrategy(),
        ]
    )
    preprocessed_df = preprocessor.transform(bindingDB_df)

    del bindingDB_df
    gc.collect()
    
    print("Creating merged dataset")
    drugbank_binding_merger = DrugBank_BindingDB_Merger()
    merged_df = drugbank_binding_merger.merge(drugbank, preprocessed_df)
    drugbank_binding_merger.save_merged(MERGED_PICKLE)

    print("Merged dataset is loaded and saved.")

Merged dataset doesn't exists.
 Creating it...
parsed_Drugbank exists...
Loading...
Load Binding DB...


C:\Users\Charaf Kamel\AppData\Local\Temp\ipykernel_3932\3397691220.py:7: DtypeWarning: Columns (8,9,10,11,12,13,15,17,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,141,142,143,146,147,148,149,150,151,153,154,155,158,159,160,161,162,163,165,166,167,170,171,172,173,174,175,177,178,179,182,183,184,185,186,187,189,190,191) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', header=0, usecols=range(df_first_row.shape[1]))


Cleaning Binding DB...
Creating merged dataset


100%|██████████| 10/10 [45:35<00:00, 273.52s/it] 


Part-1 is done
Merged dataset is loaded and saved.


In [ ]:
merged_df.head()

### Initial Cancer related filtering

In [ ]:
cancer_keywords = [
    "cancer", "tumor", "carcinoma", "neoplasm", "malignancy", "metastasis", "oncology",
    "sarcoma", "adenocarcinoma", "lymphoma", "leukemia", "myeloma", "breast cancer",
    "lung cancer", "prostate cancer", "colon cancer", "rectal cancer", "colorectal cancer",
    "skin cancer", "melanoma", "bladder cancer", "kidney cancer", "renal cancer",
    "pancreatic cancer", "ovarian cancer", "cervical cancer", "uterine cancer", "thyroid cancer",
    "liver cancer", "gastric cancer", "stomach cancer", "esophageal cancer", "brain cancer",
    "glioma", "astrocytoma", "neuroblastoma", "endometrial cancer", "dysplasia", "hyperplasia",
    "leukoplakia", "adenoma", "lesion", "metastases", "carcinoma in situ", "anaplastic",
    "fibrosarcoma", "osteosarcoma", "mesothelioma", "chemotherapy", "radiotherapy",
    "immunotherapy", "biopsy", "staging", "grading", "metastasize", "malignant", "benign",
    "tumor suppressor", "oncogene", "palliative care", "remission", "recurrence"
]
filtered_df_with_target_name = merged_df[merged_df[["Target Name", "UniProt (SwissProt) Entry Name of Target Chain"]].map(lambda x: any(keyword in str(x).lower() for keyword in cancer_keywords)).any(axis=1)]

In [ ]:
unique_cancer_related_proteins = filtered_df_with_target_name["Target Name"].unique()
print(f"number of different cancer related proteins: {len(unique_cancer_related_proteins)}")

#### Just human as source organism

In [ ]:
filtered_df_human = filtered_df_with_target_name[filtered_df_with_target_name[["Target Source Organism According to Curator or DataSource", "UniProt (SwissProt) Entry Name of Target Chain"]].map(lambda x: "human" in str(x).lower() or type(x) == float).any(axis=1)]

In [ ]:
len(filtered_df_human)

In [ ]:
mutant_keywords = [
    "mutation",
    "oncogenesis", "tumorigenesis", "apoptosis", "angiogenesis", "HER2", "BRAF", "KRAS", "EGFR",
    "BRCA1", "BRCA2", "P53", "MDM2", "CDK4", "CDK6", "V600", "G12"
]
only_mutants_df = filtered_df_human[filtered_df_human["Target Name"].apply(lambda x: any(keyword in str(x).lower() for keyword in mutant_keywords))]

In [ ]:
only_mutants_df

In [ ]:
merged_df["Matched_On"].value_counts()